In [ ]:
!pip install transformers datasets

In [ ]:
from datasets import load_dataset

# Load each dataset
mmmlu_ar_law = load_dataset("jackycedar/MMMLU_ar_law_Instruct")
arabic_law_v0 = load_dataset("kamalamarouche/Arabic_law_V0")
arabic_anss_v0 = load_dataset("kamalamarouche/Arabic_ANSS_V0")
arabic_law_v1 = load_dataset("kamalamarouche/Arabic_law_V1")

In [ ]:
print(mmmlu_ar_law)
print(arabic_law_v0)
print(arabic_anss_v0)
print(arabic_law_v1)

DatasetDict({
    train: Dataset({
        features: ['Question', 'Reply'],
        num_rows: 1642
    })
})
DatasetDict({
    train: Dataset({
        features: ['Question', 'Reply'],
        num_rows: 40
    })
})
DatasetDict({
    train: Dataset({
        features: ['Question', 'Reply'],
        num_rows: 21
    })
})
DatasetDict({
    train: Dataset({
        features: ['Question', 'Reply'],
        num_rows: 299
    })
})


In [ ]:
from datasets import concatenate_datasets

# Extract the 'train' split from each dataset
train_mmmmlu = mmmlu_ar_law["train"]
train_v0 = arabic_law_v0["train"]
train_anss = arabic_anss_v0["train"]
train_v1 = arabic_law_v1["train"]

# Combine the datasets
combined_train = concatenate_datasets([train_mmmmlu, train_v0, train_anss, train_v1])

# Print the combined dataset
print(combined_train)

Dataset({
    features: ['Question', 'Reply'],
    num_rows: 2002
})


In [ ]:
# Print the first 5 rows
for i in range(15):
    print(combined_train[i])

{'Question': 'أي عبارة مما يلي تفسر بشكل أفضل الغرض من تمييز هارت بين "كون المرء ملزمًا" و"الحصول على التزام"؟', 'Reply': 'يسلط الضوء على مفهوم القاعدة.'}
{'Question': 'غالبًا ما يُساء فهم مقولة هنري مين الشهيرة "كانت حركة المجتمعات التقدمية حتى الآن حركة من الحالة إلى العقد". كيف ذلك؟', 'Reply': 'يُساء فهمها على أنها تنبؤ.'}
{'Question': 'أيٍ من العوامل التالية ساهم بشكل كبير في إحياء القانون الطبيعي في القرن العشرين؟', 'Reply': 'الاعتراف الدولي بحقوق الإنسان بعد الحرب العالمية الثانية'}
{'Question': 'أيٍ من الانتقادات التالية لتمييز ليويلين بين الأسلوبين القانونيين الكبير والرسمي هو الأكثر إقناعًا؟', 'Reply': 'من الأمور المضلّلة تصنيف القضاة بهذه الطريقة.'}
{'Question': 'أي اقتراح مما يلي يتوافق بشكل أكبر مع ما يدعيه رولز أن يختاره أصحاب المركز الأصلي (POP) فيما يتعلق بـ "الخيرات الاجتماعية الأساسية"؟', 'Reply': 'سيختار أصحاب المركز الأصلي الثروة على حساب مجتمع رحيم.'}
{'Question': 'مذهب ________ يرى أن تعزيز كفاءة السوق يجب أن يكون الشغل الشاغل لصنع القرار القانوني.', 'Reply': 'القا

In [ ]:
combined_train.save_to_disk("combined_train")

Saving the dataset (0/1 shards):   0%|          | 0/2002 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

combined_train = load_from_disk("combined_train")

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer (e.g., AraBERT)
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

# Tokenize the dataset
def preprocess_function(examples):
    # Tokenize the 'Question' and 'Reply' columns
    model_inputs = tokenizer(examples["Question"], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["Reply"], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to the combined dataset
tokenized_dataset = combined_train.map(preprocess_function, batched=True)

In [ ]:
# Split the dataset (e.g., 90% train, 10% validation)
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load AraBERT for question answering
model = AutoModelForQuestionAnswering.from_pretrained("aubmindlab/bert-base-arabertv2")
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load CAMeLBERT for question answering
model = AutoModelForQuestionAnswering.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-da")
tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-da")

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load AraBERT for question answering
model = AutoModelForQuestionAnswering.from_pretrained("aubmindlab/bert-base-arabertv2")
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load XLM-RoBERTa for question answering
model = AutoModelForQuestionAnswering.from_pretrained("xlm-roberta-base")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from datasets import load_from_disk

# Load the combined dataset
combined_train = load_from_disk("combined_train")

In [ ]:
from transformers import AutoTokenizer

# Load the XLM-RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize the dataset
def preprocess_function(examples):
    # Tokenize the 'Question' and 'Reply' columns
    inputs = tokenizer(
        examples["Question"],
        examples["Reply"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    inputs["labels"] = inputs["input_ids"]  # Use input_ids as labels for training
    return inputs

# Apply tokenization to the combined dataset
tokenized_dataset = combined_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/2002 [00:00<?, ? examples/s]

In [ ]:
# Split the dataset (e.g., 90% train, 10% validation)
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

In [ ]:
from transformers import AutoModelForQuestionAnswering

# Load XLM-RoBERTa for question answering
model = AutoModelForQuestionAnswering.from_pretrained("xlm-roberta-base")

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./fine-tuned-xlm-roberta-law-model",  # Directory to save the model
    eval_strategy="epoch",                           # Evaluate every epoch (updated)
    learning_rate=2e-5,                              # Learning rate
    per_device_train_batch_size=4,                   # Batch size for training
    per_device_eval_batch_size=4,                    # Batch size for evaluation
    num_train_epochs=3,                              # Number of epochs
    weight_decay=0.01,                               # Weight decay
    save_total_limit=2,                              # Limit the number of saved checkpoints
    save_steps=500,                                  # Save checkpoint every 500 steps
    logging_dir="./logs",                            # Directory for logs
    logging_steps=10,                                # Log every 10 steps
    fp16=True,                                       # Enable mixed precision training
    gradient_accumulation_steps=2,                   # Accumulate gradients over 2 steps
)

In [ ]:
def preprocess_function(examples):
    # Tokenize the inputs (question + context)
    inputs = tokenizer(
        examples["Question"],
        examples["Reply"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )

    # Find the start and end positions of the answer in the tokenized input
    start_positions = []
    end_positions = []
    for i in range(len(examples["Reply"])):
        # Find the token IDs for the answer
        answer_tokens = tokenizer(
            examples["Reply"][i],
            add_special_tokens=False,
            return_tensors="pt",
        )["input_ids"][0]

        # Find the start and end positions of the answer in the input_ids
        start_positions.append(tokenizer(examples["Question"][i], examples["Reply"][i]).input_ids.index(answer_tokens[0]))
        end_positions.append(tokenizer(examples["Question"][i], examples["Reply"][i]).input_ids.index(answer_tokens[-1]))

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Apply preprocessing to the dataset
tokenized_dataset = combined_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/2002 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (586 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer

# Load the dataset
combined_train = load_from_disk("combined_train")

# Load the XLM-RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize the dataset and add start/end positions
def preprocess_function(examples):
    # Tokenize the inputs (question + context)
    inputs = tokenizer(
        examples["Question"],
        examples["Reply"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )

    # Find the start and end positions of the answer in the tokenized input
    start_positions = []
    end_positions = []
    for i in range(len(examples["Reply"])):
        # Tokenize the answer
        answer_tokens = tokenizer(
            examples["Reply"][i],
            add_special_tokens=False,
            return_tensors="pt",
        )["input_ids"][0]

        # Find the start and end positions of the answer in the input_ids
        input_ids = inputs["input_ids"][i]
        start_positions.append(input_ids.index(answer_tokens[0]))
        end_positions.append(input_ids.index(answer_tokens[-1]))

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Apply preprocessing to the dataset
tokenized_dataset = combined_train.map(preprocess_function, batched=True)

# Split the dataset
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

Map:   0%|          | 0/2002 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (563 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
from transformers import AutoModelForQuestionAnswering

# Load XLM-RoBERTa for question answering
model = AutoModelForQuestionAnswering.from_pretrained("xlm-roberta-base")

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./fine-tuned-xlm-roberta-law-model",  # Directory to save the model
    eval_strategy="epoch",                           # Evaluate every epoch
    learning_rate=2e-5,                              # Learning rate
    per_device_train_batch_size=4,                   # Batch size for training
    per_device_eval_batch_size=4,                    # Batch size for evaluation
    num_train_epochs=3,                              # Number of epochs
    weight_decay=0.01,                               # Weight decay
    save_total_limit=2,                              # Limit the number of saved checkpoints
    save_steps=500,                                  # Save checkpoint every 500 steps
    logging_dir="./logs",                            # Directory for logs
    logging_steps=10,                                # Log every 10 steps
    fp16=True,                                       # Enable mixed precision training
    gradient_accumulation_steps=2,                   # Accumulate gradients over 2 steps
)

In [ ]:
from transformers import Trainer

# Create the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

<ipython-input-69-35ddfadfbbda>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.762300,0.419846
2,0.158300,0.121563


TrainOutput(global_step=675, training_loss=0.7471546625207972, metrics={'train_runtime': 311.9069, 'train_samples_per_second': 17.322, 'train_steps_per_second': 2.164, 'total_flos': 1407344331780096.0, 'train_loss': 0.7471546625207972, 'epoch': 2.988913525498891})

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer.save_pretrained("./fine-tuned-xlm-roberta-law-model")

('./fine-tuned-xlm-roberta-law-model/tokenizer_config.json',
 './fine-tuned-xlm-roberta-law-model/special_tokens_map.json',
 './fine-tuned-xlm-roberta-law-model/sentencepiece.bpe.model',
 './fine-tuned-xlm-roberta-law-model/added_tokens.json',
 './fine-tuned-xlm-roberta-law-model/tokenizer.json')

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.12156341969966888, 'eval_runtime': 1.678, 'eval_samples_per_second': 119.783, 'eval_steps_per_second': 30.393, 'epoch': 2.988913525498891}


In [ ]:
# Step 1: Load the fine-tuned model and tokenizer
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

model = AutoModelForQuestionAnswering.from_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-xlm-roberta-law-model")

# Step 2: Define a function to generate answers
def generate_answer(question, context):
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)
    answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1], skip_special_tokens=True)
    return answer

# Step 3: Evaluate the chatbot on law-related questions
context = """
القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير.
يعتمد القانون الطبيعي على فكرة أن هناك قواعد أخلاقية يمكن اكتشافها بالعقل البشري وتطبيقها على جميع المجتمعات.
من أبرز الفلاسفة الذين دافعوا عن القانون الطبيعي الفيلسوف الروماني شيشرون، الذي قال: "القانون الحقيقي هو العقل الصحيح المتفق مع الطبيعة".
"""

questions = [
    "ما هو القانون الطبيعي؟",
    "من هو الفيلسوف الروماني الذي دافع عن القانون الطبيعي؟",
    "ما هي الفكرة الأساسية للقانون الطبيعي؟",
]

for question in questions:
    answer = generate_answer(question, context)
    print(f"السؤال: {question}")
    print(f"الإجابة: {answer}")
    print("-" * 50)

السؤال: ما هو القانون الطبيعي؟
الإجابة: القانون الطبيعي؟ القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير. يعتمد القانون الطبيعي على فكرة أن هناك قواعد أخلاقية يمكن اكتشافها بالعقل البشري وتطبيقها على جميع المجتمعات. من أبرز الفلاسفة الذين دافعوا عن القانون الطبيعي الفيلسوف الروماني شيشرون، الذي قال: "القانون الحقيقي هو العقل الصحيح المتفق مع الطبيعة".
--------------------------------------------------
السؤال: من هو الفيلسوف الروماني الذي دافع عن القانون الطبيعي؟
الإجابة: القانون الطبيعي؟ القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير. يعتمد القانون الطبيعي على فكرة أن هناك قواعد أخلاقية يمكن اكتشافها بالعقل البشري وتطبيقها على جميع المجتمعات. من أبرز الفلاسفة الذين دافعوا عن القانون الطبيعي الفيلسوف الروماني شيشرون، الذي قال: "القانون الحقيقي هو العقل الصحيح المتفق مع الطبيعة".
--------------------------------------------------
السؤال: ما هي الفكرة الأساسية للقانون الطبيعي؟
الإجابة: القانون ا

In [ ]:
# Step 1: Load the fine-tuned model and tokenizer
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

model = AutoModelForQuestionAnswering.from_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-xlm-roberta-law-model")

def generate_answer(question, context):
    # Tokenize the input (question + context)
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # Find the start and end positions of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Ensure the end index is greater than or equal to the start index
    if end_index < start_index:
        end_index = start_index

    # Decode the answer
    answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1], skip_special_tokens=True)
    return answer
# Step 3: Evaluate the chatbot on law-related questions
# Define a context (e.g., a paragraph about law)
context = """
القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير.
يعتمد القانون الطبيعي على فكرة أن هناك قواعد أخلاقية يمكن اكتشافها بالعقل البشري وتطبيقها على جميع المجتمعات.
من أبرز الفلاسفة الذين دافعوا عن القانون الطبيعي الفيلسوف الروماني شيشرون، الذي قال: "القانون الحقيقي هو العقل الصحيح المتفق مع الطبيعة".
"""

# Define a list of questions
questions = [
    "ما هو القانون الطبيعي؟",
    "من هو الفيلسوف الروماني الذي دافع عن القانون الطبيعي؟",
    "ما هي الفكرة الأساسية للقانون الطبيعي؟",
]

# Evaluate the chatbot on the questions
for question in questions:
    answer = generate_answer(question, context)
    print(f"السؤال: {question}")
    print(f"الإجابة: {answer}")
    print("-" * 50)

السؤال: ما هو القانون الطبيعي؟
الإجابة: القانون الطبيعي؟ القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير. يعتمد القانون الطبيعي على فكرة أن هناك قواعد أخلاقية يمكن اكتشافها بالعقل البشري وتطبيقها على جميع المجتمعات. من أبرز الفلاسفة الذين دافعوا عن القانون الطبيعي الفيلسوف الروماني شيشرون، الذي قال: "القانون الحقيقي هو العقل الصحيح المتفق مع الطبيعة".
--------------------------------------------------
السؤال: من هو الفيلسوف الروماني الذي دافع عن القانون الطبيعي؟
الإجابة: القانون الطبيعي؟ القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير. يعتمد القانون الطبيعي على فكرة أن هناك قواعد أخلاقية يمكن اكتشافها بالعقل البشري وتطبيقها على جميع المجتمعات. من أبرز الفلاسفة الذين دافعوا عن القانون الطبيعي الفيلسوف الروماني شيشرون، الذي قال: "القانون الحقيقي هو العقل الصحيح المتفق مع الطبيعة".
--------------------------------------------------
السؤال: ما هي الفكرة الأساسية للقانون الطبيعي؟
الإجابة: القانون ا

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer.save_pretrained("./fine-tuned-xlm-roberta-law-model")

('./fine-tuned-xlm-roberta-law-model/tokenizer_config.json',
 './fine-tuned-xlm-roberta-law-model/special_tokens_map.json',
 './fine-tuned-xlm-roberta-law-model/sentencepiece.bpe.model',
 './fine-tuned-xlm-roberta-law-model/added_tokens.json',
 './fine-tuned-xlm-roberta-law-model/tokenizer.json')

In [ ]:
!pip install flask

In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

app = Flask(__name__)

# Load the fine-tuned model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-xlm-roberta-law-model")

@app.route("/ask", methods=["POST"])
def ask():
    data = request.json
    question = data["question"]
    context = data["context"]

    # Tokenize the input (question + context)
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # Find the start and end positions of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Ensure the end index is greater than or equal to the start index
    if end_index < start_index:
        end_index = start_index

    # Decode the answer
    answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1], skip_special_tokens=True)

    return jsonify({"reply": answer})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2hmrHVylhqTORRlbpETeczeiNH6_4hVi5MX6yxkgUpfmipZTp")  # Replace with your authtoken

# Start ngrok and create a public URL
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

# Run the Flask app
app.run(host="0.0.0.0", port=5000)

Public URL: NgrokTunnel: "https://5c0a-34-125-9-27.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!curl -X POST https://13b3-34-125-9-27.ngrok-free.app/ask \
-H "Content-Type: application/json" \
-d '{"question": "ما هو القانون الطبيعي؟", "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."}'

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

app = Flask(__name__)

# Load the fine-tuned model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-xlm-roberta-law-model")

@app.route("/ask", methods=["POST"])
def ask():
    data = request.json
    question = data["question"]
    context = data["context"]

    # Tokenize the input (question + context)
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # Find the start and end positions of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Ensure the end index is greater than or equal to the start index
    if end_index < start_index:
        end_index = start_index

    # Decode the answer
    answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1], skip_special_tokens=True)

    return jsonify({"reply": answer})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2hmrHVylhqTORRlbpETeczeiNH6_4hVi5MX6yxkgUpfmipZTp")  # Replace with your authtoken

# Start ngrok and create a public URL
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://41f1-34-125-9-27.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
!curl -X POST https://41f1-34-125-9-27.ngrok-free.app/ask \
-H "Content-Type: application/json" \
-d '{"question": "ما هو القانون الطبيعي؟", "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."}'

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
import requests

# Define the API URL
api_url = "https://41f1-34-125-9-27.ngrok-free.app/ask"

# Define the request payload
payload = {
    "question": "ما هو القانون الطبيعي؟",
    "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."
}

# Send the POST request
response = requests.post(api_url, json=payload)

# Print the response
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

app = Flask(__name__)

# Load the fine-tuned model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-xlm-roberta-law-model")

@app.route("/ask", methods=["POST"])
def ask():
    data = request.json
    question = data["question"]
    context = data["context"]

    # Tokenize the input (question + context)
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # Find the start and end positions of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Ensure the end index is greater than or equal to the start index
    if end_index < start_index:
        end_index = start_index

    # Decode the answer
    answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1], skip_special_tokens=True)

    return jsonify({"reply": answer})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2hmrHVylhqTORRlbpETeczeiNH6_4hVi5MX6yxkgUpfmipZTp")  # Replace with your authtoken

# Start ngrok and create a public URL
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://96b5-34-125-9-27.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
!curl -X POST https://41f1-34-125-9-27.ngrok-free.app/ask \
-H "Content-Type: application/json" \
-d '{"question": "ما هو القانون الطبيعي؟", "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."}'

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

app = Flask(__name__)

# Load the fine-tuned model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-xlm-roberta-law-model")

@app.route("/ask", methods=["POST"])
def ask():
    data = request.json
    question = data["question"]
    context = data["context"]

    # Tokenize the input (question + context)
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # Find the start and end positions of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Ensure the end index is greater than or equal to the start index
    if end_index < start_index:
        end_index = start_index

    # Decode the answer
    answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1], skip_special_tokens=True)

    return jsonify({"reply": answer})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2hmrHVylhqTORRlbpETeczeiNH6_4hVi5MX6yxkgUpfmipZTp")  # Replace with your authtoken

# Start ngrok and create a public URL
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://7800-34-125-9-27.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
!curl -X POST https://96b5-34-125-9-27.ngrok-free.app/ask \
-H "Content-Type: application/json" \
-d '{"question": "ما هو القانون الطبيعي؟", "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."}'

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

app = Flask(__name__)

# Load the fine-tuned model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-xlm-roberta-law-model")

@app.route("/ask", methods=["POST"])
def ask():
    data = request.json
    question = data["question"]
    context = data["context"]

    # Tokenize the input (question + context)
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # Find the start and end positions of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Ensure the end index is greater than or equal to the start index
    if end_index < start_index:
        end_index = start_index

    # Decode the answer
    answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1], skip_special_tokens=True)

    return jsonify({"reply": answer})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2hmrHVylhqTORRlbpETeczeiNH6_4hVi5MX6yxkgUpfmipZTp")  # Replace with your authtoken

# Start ngrok and create a public URL
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://fe00-34-125-9-27.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from flask import Flask
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2hmrHVylhqTORRlbpETeczeiNH6_4hVi5MX6yxkgUpfmipZTp")  # Replace with your authtoken

# Create a Flask app
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

# Start the Flask app
if __name__ == '__main__':
    # Start ngrok and create a public URL
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)

    # Run the Flask app
    app.run(host='0.0.0.0', port=5000)

Public URL: NgrokTunnel: "https://8d0f-34-125-9-27.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 20:46:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 20:46:03] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
!curl -X POST https://96b5-34-125-9-27.ngrok-free.app/ask \
-H "Content-Type: application/json" \
-d '{"question": "ما هو القانون الطبيعي؟", "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."}'

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import time

# Set your ngrok authtoken
ngrok.set_auth_token("2hmrHVylhqTORRlbpETeczeiNH6_4hVi5MX6yxkgUpfmipZTp")  # Replace with your authtoken

# Create a Flask app
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/ask', methods=['POST'])
def ask_question():
    data = request.json
    question = data.get('question')
    context = data.get('context')
    # Process the question and context (this is just a placeholder)
    response = f"Question: {question}, Context: {context}"
    return jsonify({"response": response})

# Start the Flask app
if __name__ == '__main__':
    # Start ngrok and create a public URL
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)

    # Run the Flask app
    app.run(host='0.0.0.0', port=5000)

Public URL: NgrokTunnel: "https://94f0-34-125-9-27.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 20:48:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 20:48:44] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
curl -X POST https://8d0f-34-125-9-27.ngrok-free.app/ask \
-H "Content-Type: application/json" \
-d '{"question": "ما هو القانون الطبيعي؟", "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."}'

SyntaxError: invalid decimal literal (<ipython-input-110-05143cddd067>, line 1)

In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2hmrHVylhqTORRlbpETeczeiNH6_4hVi5MX6yxkgUpfmipZTp")  # Replace with your authtoken

# Create a Flask app
app = Flask(__name__)

# Load the fine-tuned model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("./fine-tuned-xlm-roberta-law-model")
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-xlm-roberta-law-model")

@app.route("/ask", methods=["POST"])
def ask():
    data = request.json
    question = data["question"]
    context = data["context"]

    # Tokenize the input (question + context)
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # Find the start and end positions of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Ensure the end index is greater than or equal to the start index
    if end_index < start_index:
        end_index = start_index

    # Decode the answer
    answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1], skip_special_tokens=True)

    return jsonify({"reply": answer})

# Start the Flask app
if __name__ == "__main__":
    # Start ngrok and create a public URL
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)

    # Run the Flask app
    app.run(host="0.0.0.0", port=5000)

Public URL: NgrokTunnel: "https://0692-34-125-9-27.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 20:50:01] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 20:50:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 20:51:48] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 20:54:34] "GET / HTTP/1.1" 404 -


In [ ]:
!curl https://0692-34-125-9-27.ngrok-free.app/

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
!curl -X POST https://0692-34-125-9-27.ngrok-free.app/ask \
-H "Content-Type: application/json" \
-d '{"question": "ما هو القانون الطبيعي؟", "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."}'

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
!curl -X POST https://018d-34-125-9-27.ngrok-free.app/ask \
-H "Content-Type: application/json" \
-d '{"question": "ما هو القانون الطبيعي؟", "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."}'

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
!curl https://0692-34-125-9-27.ngrok-free.app/

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
!curl https://0692-34-125-9-27.ngrok-free.app/

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [ ]:
!curl -X POST https://0692-34-125-9-27.ngrok-free.app/ask \
-H "Content-Type: application/json" \
-d '{"question": "ما هو القانون الطبيعي؟", "context": "القانون الطبيعي هو مجموعة من المبادئ الأخلاقية والقانونية التي تعتبر عالمية وغير قابلة للتغيير."}'

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex